In [7]:
import logging
import os
import shutil
import subprocess
import argparse

import torch
from flask import Flask, jsonify, request
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings,HuggingFaceEmbeddings

from run_inference import load_model,retrieval_qa_pipeline
from prompt_template_utils import get_prompt_template

# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma, FAISS
from werkzeug.utils import secure_filename

from constants import CHROMA_SETTINGS, EMBEDDING_MODEL_NAME, PERSIST_DIRECTORY, MODEL_ID, MODEL_BASENAME

In [9]:
print(PERSIST_DIRECTORY)

../VECTOR_DBS


In [10]:
## setting up the args - these can be made runtime args
device_type='cpu'
show_sources=True
use_history=True
save_qa=True
promptTemplate_type="llama"

logging.info(f"Display Source Documents set to: {show_sources}")
print(f"Display Source Documents set to: {show_sources}")
logging.info(f"Display Use History set to: {use_history}")
print(f"Display Use History set to: {use_history}")
logging.info(f"Display promptTemplate_type set to: {promptTemplate_type}")
logging.info(f"Display Save QA set to: {save_qa}")


Display Source Documents set to: True
Display Use History set to: True


In [11]:
QA = retrieval_qa_pipeline(use_history, promptTemplate_type=promptTemplate_type,device_type=device_type)

load INSTRUCTOR_Transformer
max_seq_length  512
Loading Model: TheBloke/Llama-2-7b-Chat-GGUF, on: cpu


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [12]:
app = Flask(__name__)

In [13]:
@app.route("/api/prompt_route", methods=["GET", "POST"])
def prompt_route():
    global QA
    user_prompt = request.form.get("user_prompt")
    #user_prompt = ' Magnesium has an atomic number of 12. Which of the following statements is true of a neutral magnesium atom?'
    print(f'User Prompt: {user_prompt}')
    if user_prompt:
        #print(f'User Prompt: {user_prompt}')
        # Get the answer from the chain
        res = QA(user_prompt)
        answer, docs = res["result"], res["source_documents"]

        prompt_response_dict = {
            "Prompt": user_prompt,
            "Answer": answer,
        }
        
        print(answer)
        
        prompt_response_dict["Sources"] = []
        for document in docs:
            prompt_response_dict["Sources"].append(
                (os.path.basename(str(document.metadata["source"])), str(document.page_content))
            )

        return jsonify(prompt_response_dict), 200
    else:
        return "No user prompt received", 400

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--port", type=int, default=5110, help="Port to run the API on. Defaults to 5110.")
    parser.add_argument(
        "--host",
        type=str,
        default="127.0.0.1",
        help="Host to run the UI on. Defaults to 127.0.0.1. "
        "Set to 0.0.0.0 to make the UI externally "
        "accessible from other devices.",
    )
    args = parser.parse_args()

    app.run(debug=False, host=args.host, port=args.port)


In [14]:
host = '127.0.0.1'
port='5110'
app.run(debug=False, host=host, port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5110
INFO:werkzeug:Press CTRL+C to quit
